# **"I love llamas"를 "Ik hou van lama's"로 번역하는 예제**



---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.



---



## **1. RNN(Recurrent Neural Network) 아키텍처 사용 : Seq2Seq**
- (실행을 위해 앞 코드에서 추가한 사항)
    - **어휘 사전** : Vocabulary 클래스 → 단어-인덱스 매핑 자동화
    - **특수 토큰** : `<SOS>`, `<EOS>`, `<PAD>`, `<UNK>` →문장 경계 및 패딩 처리
    - **Teacher Forcing** : 확률적 적용 →훈련 안정성 향상
    - **드롭아웃** : 0.2 비율 적용 → 과적합 방지

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
import random

# 1. 데이터 준비 및 전처리
class Vocabulary:
    def __init__(self):
        self.word2idx = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
        self.idx2word = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>'}
        self.word_count = 4

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.word_count
            self.idx2word[self.word_count] = word
            self.word_count += 1

    def __len__(self):
        return self.word_count

def create_datasets():
    """영어-네덜란드어 번역 데이터셋 생성"""
    # 간단한 훈련 데이터 (실제로는 더 많은 데이터가 필요)
    english_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "I read books",
        "I watch movies",
        "I play games",
        "I write code",
        "I love programming",
        "I study machine learning"
    ]

    dutch_sentences = [
        "Ik hou van lama's",
        "Ik hou van katten",
        "Ik eet appels",
        "Ik drink water",
        "Ik lees boeken",
        "Ik kijk naar films",
        "Ik speel spelletjes",
        "Ik schrijf code",
        "Ik hou van programmeren",
        "Ik studeer machine learning"
    ]

    return english_sentences, dutch_sentences

def preprocess_data(english_sentences, dutch_sentences):
    """데이터 전처리 및 어휘 사전 생성"""
    en_vocab = Vocabulary()
    nl_vocab = Vocabulary()

    # 어휘 사전 구축
    for sentence in english_sentences:
        for word in sentence.lower().split():
            en_vocab.add_word(word)

    for sentence in dutch_sentences:
        for word in sentence.lower().split():
            nl_vocab.add_word(word)

    return en_vocab, nl_vocab

def sentence_to_indices(sentence, vocab, max_len=10):
    """문장을 인덱스 시퀀스로 변환"""
    words = sentence.lower().split()
    indices = [vocab.word2idx.get(word, vocab.word2idx['<UNK>']) for word in words]

    # 최대 길이 제한 (EOS 토큰 공간 확보)
    if len(indices) >= max_len:
        indices = indices[:max_len-1]

    # 패딩 추가
    while len(indices) < max_len:
        indices.append(vocab.word2idx['<PAD>'])

    return indices

# 2. 모델 정의 (기존 코드 개선)
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hidden, cell):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output)
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.fc.out_features

        # 인코딩
        hidden, cell = self.encoder(source)

        # 디코딩 결과 저장
        outputs = torch.zeros(batch_size, target_len, target_vocab_size)

        # 첫 번째 입력은 <SOS> 토큰
        input_token = target[:, 0:1]

        for i in range(1, target_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, i:i+1] = output

            # Teacher forcing 적용
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(-1)
            input_token = target[:, i:i+1] if teacher_force else top1

        return outputs

# 3. 훈련 함수
def train_model(model, train_data, en_vocab, nl_vocab, num_epochs=100):
    """모델 훈련"""
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # 패딩 무시
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    max_len = 10

    for epoch in range(num_epochs):
        total_loss = 0

        for en_sentence, nl_sentence in train_data:
            # 데이터 준비 - 길이 통일
            en_indices = sentence_to_indices(en_sentence, en_vocab, max_len)
            nl_words = sentence_to_indices(nl_sentence, nl_vocab, max_len-1)  # SOS/EOS 공간 확보

            # 디코더 입력: <SOS> + 문장 (마지막 제외)
            nl_input = [nl_vocab.word2idx['<SOS>']] + nl_words[:-1]
            # 타겟: 문장 + <EOS>
            nl_target = nl_words + [nl_vocab.word2idx['<EOS>']]

            # 길이 맞추기
            nl_input = nl_input[:max_len]
            nl_target = nl_target[:max_len]

            # 패딩으로 길이 통일
            while len(nl_input) < max_len:
                nl_input.append(nl_vocab.word2idx['<PAD>'])
            while len(nl_target) < max_len:
                nl_target.append(nl_vocab.word2idx['<PAD>'])

            en_tensor = torch.LongTensor([en_indices])
            nl_tensor = torch.LongTensor([nl_input])
            target_tensor = torch.LongTensor([nl_target])

            # 순전파
            optimizer.zero_grad()
            output = model(en_tensor, nl_tensor)

            # 손실 계산 - 차원 맞추기
            output = output.reshape(-1, output.shape[-1])  # [batch*seq, vocab]
            target_tensor = target_tensor.reshape(-1)       # [batch*seq]
            loss = criterion(output, target_tensor)

            # 역전파
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 20 == 0:
            avg_loss = total_loss / len(train_data)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 4. 번역 함수
def translate(model, sentence, en_vocab, nl_vocab, max_len=10):
    """문장 번역"""
    model.eval()

    with torch.no_grad():
        # 입력 문장 전처리
        en_indices = sentence_to_indices(sentence, en_vocab)
        en_tensor = torch.LongTensor([en_indices])

        # 인코딩
        hidden, cell = model.encoder(en_tensor)

        # 디코딩
        result = []
        input_token = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

        for _ in range(max_len):
            output, hidden, cell = model.decoder(input_token, hidden, cell)
            predicted_id = output.argmax(-1).item()

            if predicted_id == nl_vocab.word2idx['<EOS>']:
                break

            if predicted_id != nl_vocab.word2idx['<PAD>']:
                word = nl_vocab.idx2word[predicted_id]
                result.append(word)

            input_token = torch.LongTensor([[predicted_id]])

        return ' '.join(result)

# 5. 메인 실행 코드
def main():
    """메인 실행 함수"""
    print("=" * 50)
    print("영어-네덜란드어 Seq2Seq 번역 모델")
    print("=" * 50)

    # 데이터 준비
    english_sentences, dutch_sentences = create_datasets()
    en_vocab, nl_vocab = preprocess_data(english_sentences, dutch_sentences)

    print(f"영어 어휘 수: {len(en_vocab)}")
    print(f"네덜란드어 어휘 수: {len(nl_vocab)}")
    print()

    # 훈련 데이터
    train_data = list(zip(english_sentences, dutch_sentences))

    # 모델 생성
    embed_size = 128
    hidden_size = 256

    encoder = Encoder(len(en_vocab), embed_size, hidden_size)
    decoder = Decoder(len(nl_vocab), embed_size, hidden_size)
    model = Seq2Seq(encoder, decoder)

    print("모델 구조:")
    print(f"Encoder: 어휘 크기 {len(en_vocab)}, 임베딩 {embed_size}, 은닉 {hidden_size}")
    print(f"Decoder: 어휘 크기 {len(nl_vocab)}, 임베딩 {embed_size}, 은닉 {hidden_size}")
    print()

    # 모델 훈련
    print("모델 훈련 시작...")
    train_model(model, train_data, en_vocab, nl_vocab, num_epochs=200)
    print()

    # 번역 테스트
    print("번역 테스트:")
    print("-" * 30)

    test_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water"
    ]

    for sentence in test_sentences:
        translation = translate(model, sentence, en_vocab, nl_vocab)
        print(f"영어: {sentence}")
        print(f"네덜란드어: {translation}")
        print()

    # 새로운 문장 번역 (목표 문장)
    print("=" * 30)
    print("목표 번역:")
    target_sentence = "I love llamas"
    translation = translate(model, target_sentence, en_vocab, nl_vocab)
    print(f"영어: {target_sentence}")
    print(f"네덜란드어 번역: {translation}")
    print(f"정답: Ik hou van lama's")

# 6. 모델 성능 분석
def analyze_model_performance():
    """모델 성능 분석"""
    print("\n" + "=" * 40)
    print("모델 성능 분석")
    print("=" * 40)

    print("1. Seq2Seq 모델의 특징:")
    print("   - 인코더: 입력 문장을 고정 크기 벡터로 압축")
    print("   - 디코더: 압축된 벡터로부터 출력 문장 생성")
    print("   - Teacher Forcing: 훈련 시 정답을 다음 입력으로 사용")

    print("\n2. 한계점:")
    print("   - 긴 문장에서 정보 손실 발생 가능")
    print("   - 입력 문장의 모든 정보를 하나의 벡터에 압축")
    print("   - 단어 순서가 다른 언어쌍에서 성능 제한")

    print("\n3. 개선 방안:")
    print("   - Attention 메커니즘 추가")
    print("   - 더 많은 훈련 데이터 사용")
    print("   - Transformer 모델 도입")

if __name__ == "__main__":
    main()
    analyze_model_performance()

영어-네덜란드어 Seq2Seq 번역 모델
영어 어휘 수: 25
네덜란드어 어휘 수: 26

모델 구조:
Encoder: 어휘 크기 25, 임베딩 128, 은닉 256
Decoder: 어휘 크기 26, 임베딩 128, 은닉 256

모델 훈련 시작...
Epoch [20/200], Loss: 0.8251
Epoch [40/200], Loss: 0.7602
Epoch [60/200], Loss: 0.7377
Epoch [80/200], Loss: 0.7356
Epoch [100/200], Loss: 0.7348
Epoch [120/200], Loss: 0.7342
Epoch [140/200], Loss: 0.7339
Epoch [160/200], Loss: 0.7337
Epoch [180/200], Loss: 0.7336
Epoch [200/200], Loss: 0.7335

번역 테스트:
------------------------------
영어: I love llamas
네덜란드어: hou van lama's lama's lama's lama's

영어: I like cats
네덜란드어: hou van katten katten katten

영어: I eat apples
네덜란드어: eet appels appels appels films

영어: I drink water
네덜란드어: drink water water water

목표 번역:
영어: I love llamas
네덜란드어 번역: hou van lama's lama's lama's lama's
정답: Ik hou van lama's

모델 성능 분석
1. Seq2Seq 모델의 특징:
   - 인코더: 입력 문장을 고정 크기 벡터로 압축
   - 디코더: 압축된 벡터로부터 출력 문장 생성
   - Teacher Forcing: 훈련 시 정답을 다음 입력으로 사용

2. 한계점:
   - 긴 문장에서 정보 손실 발생 가능
   - 입력 문장의 모든 정보를 하나의 벡터에 압축
   - 단어 순서가 다른 언

- 문제점
    - 네덜란드어 번역: hou van lama's lama's lama's lama's
    - lama's 가 반복적으로 나옴.
    - 입력 문장을 고정 크기 벡터인해 장기 기억 손실이 발생함
    - Decoder가 "I love llamas"까지 번역한 후, "그다음엔 뭘 번역해야 하지?"라고 문맥 벡터를 봤을 때, 이미 정보가 손실되어 "원래 문장이 끝났는지", "다음 단어가 무엇인지" 알 수 없게 됩니다.(모델은 "내가 방금 전에 'lama's'를 말했다"는 사실은 기억하지만, "내가 아주 오래전에 'I love'를 말했다"는 사실은 잊어버리게 됩니다.)




---



## **2. Attention 아키텍처 적용 : Seq2Seq + Attention**

|개선 영역|기존 Seq2Seq|Attention Seq2Seq|개선효과|
|---|---|---|---|
|인코더|단방향 LSTM|양방향 LSTM|문맥 정보 풍부화|
|정보 전달|마지막 상태만 사용|모든 상태 활용|정보 손실 방지|
|디코더| 입력이전 출력 + 고정 컨텍스트|이전 출력 + 동적 컨텍스트|적응적 번역|
|메모리|O(1) 컨텍스트|O(n) 모든 상태 저장|긴 문장 처리 가능|
|해석 가능성|블랙박스|Attention 가중치 시각화|번역 과정 이해|

- **기존 Seq2Seq와 차이점(encoder, decoder, Seq2Seq Class)**
    - **Encoder vs ImprovedEncode**r Encoder는 마지막 시점의 은닉 상태(context vector)만 반환하는 반면, ImprovedEncoder는 어텐션 메커니즘에 사용될 인코더의 **모든 시점 출력값**(outputs)과 **패딩 마스크**(mask)를 추가로 반환합니다.
    - **Decoder vs ImprovedDecode**r Decoder는 LSTM의 은닉 상태만으로 예측을 수행하지만, ImprovedDecoder는 **어텐션*8(Attention)을 도입하여 인코더의 모든 출력(context)과 현재 LSTM 출력을 **결합**(concatenate)한 뒤 최종 예측을 수행합니다.
    - **Seq2Seq vs ImprovedSeq2Seq** Seq2Seq는 인코더의 마지막 은닉 상태만 디코더의 초기 상태로 전달하는 반면, ImprovedSeq2Seq는 어텐션을 위해 인코더의 모든 출력(encoder_outputs)과 마스크를 디코더의 매 시점에 전달하며, **어텐션 가중치(attention_weights)를 함께 반환**합니다.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import Counter
import random

# 시드 설정으로 재현 가능한 결과 보장
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 1. 데이터 준비 및 전처리
class Vocabulary:
    def __init__(self):
        self.word2idx = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
        self.idx2word = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>'}
        self.word_count = 4

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.word_count
            self.idx2word[self.word_count] = word
            self.word_count += 1

    def __len__(self):
        return self.word_count

def create_datasets():
    """영어-네덜란드어 번역 데이터셋 생성"""
    english_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "I read books",
        "I watch movies",
        "I play games",
        "I write code",
        "I love programming",
        "I study machine learning"
    ]

    dutch_sentences = [
        "Ik hou van lama's",
        "Ik hou van katten",
        "Ik eet appels",
        "Ik drink water",
        "Ik lees boeken",
        "Ik kijk naar films",
        "Ik speel spelletjes",
        "Ik schrijf code",
        "Ik hou van programmeren",
        "Ik studeer machine learning"
    ]

    return english_sentences, dutch_sentences

def preprocess_data(english_sentences, dutch_sentences):
    """데이터 전처리 및 어휘 사전 생성"""
    en_vocab = Vocabulary()
    nl_vocab = Vocabulary()

    for sentence in english_sentences:
        for word in sentence.lower().split():
            en_vocab.add_word(word)

    for sentence in dutch_sentences:
        for word in sentence.lower().split():
            nl_vocab.add_word(word)

    return en_vocab, nl_vocab

def sentence_to_indices(sentence, vocab, max_len=10):
    """문장을 인덱스 시퀀스로 변환"""
    words = sentence.lower().split()
    indices = [vocab.word2idx.get(word, vocab.word2idx['<UNK>']) for word in words]

    if len(indices) >= max_len:
        indices = indices[:max_len-1]

    while len(indices) < max_len:
        indices.append(vocab.word2idx['<PAD>'])

    return indices


# 2. 수정된 Attention 메커니즘
class ImprovedAttention(nn.Module):
    """개선된 Dot-Product Attention (Luong-style),
        - 초기 초기 RNN 기반 Seq2Seq 모델의 어텐션 메커니즘 종류, Bahdanau-style"""

    def __init__(self, hidden_size):
        super(ImprovedAttention, self).__init__()
        self.hidden_size = hidden_size

        # Luong Attention - 더 간단하고 효과적
        self.W_a = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        """
        Args:
            decoder_hidden: [batch_size, hidden_size] - 현재 디코더 상태
            encoder_outputs: [batch_size, seq_len, hidden_size] - 인코더 모든 출력
            mask: [batch_size, seq_len] - 패딩 마스크
        """
        batch_size, seq_len, hidden_size = encoder_outputs.size()

        # Luong Attention: score = decoder_hidden^T * W_a * encoder_outputs
        # decoder_hidden: [batch_size, hidden_size]
        # W_a(encoder_outputs): [batch_size, seq_len, hidden_size]

        transformed_encoder = self.W_a(encoder_outputs)  # [batch_size, seq_len, hidden_size]
        decoder_hidden = decoder_hidden.unsqueeze(2)     # [batch_size, hidden_size, 1]

        # 배치 행렬 곱셈으로 attention score 계산
        attention_scores = torch.bmm(transformed_encoder, decoder_hidden)  # [batch_size, seq_len, 1]
        attention_scores = attention_scores.squeeze(2)  # [batch_size, seq_len]

        # 패딩 마스킹
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)

        # Softmax로 정규화
        attention_weights = F.softmax(attention_scores, dim=1)  # [batch_size, seq_len]

        # Context vector 계산
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)  # [batch_size, 1, hidden_size]
        context_vector = context_vector.squeeze(1)  # [batch_size, hidden_size]

        return attention_weights, context_vector

# 3. 개선된 Encoder
class ImprovedEncoder(nn.Module):
    """개선된 인코더"""

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.dropout = nn.Dropout(dropout)

        # 단방향 LSTM 사용 (더 안정적)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x):
        # 패딩 마스크 생성
        mask = (x != 0).float()  # [batch_size, seq_len]

        embedded = self.dropout(self.embedding(x))  # [batch_size, seq_len, embed_size]

        # LSTM forward pass
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs: [batch_size, seq_len, hidden_size]
        # hidden: [num_layers, batch_size, hidden_size]

        return outputs, (hidden, cell), mask

# 4. 개선된 Decoder
class ImprovedDecoder(nn.Module):
    """개선된 디코더"""

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.attention = ImprovedAttention(hidden_size)
        self.dropout = nn.Dropout(dropout)

        # LSTM: embedding만 입력으로 사용
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # 출력층: LSTM output + context vector 결합
        self.output_projection = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, input_token, hidden, cell, encoder_outputs, encoder_mask):
        batch_size = input_token.size(0)

        # 임베딩
        embedded = self.dropout(self.embedding(input_token))  # [batch_size, 1, embed_size]

        # LSTM
        lstm_output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # lstm_output: [batch_size, 1, hidden_size]

        # Attention 계산
        decoder_hidden = lstm_output.squeeze(1)  # [batch_size, hidden_size]
        attention_weights, context_vector = self.attention(
            decoder_hidden, encoder_outputs, encoder_mask
        )

        # LSTM 출력과 컨텍스트 벡터 결합
        combined = torch.cat([decoder_hidden, context_vector], dim=1)  # [batch_size, hidden_size * 2]

        # 최종 출력
        output = self.output_projection(combined)  # [batch_size, vocab_size]
        output = output.unsqueeze(1)  # [batch_size, 1, vocab_size]

        return output, hidden, cell, attention_weights

# 5. 개선된 Seq2Seq 모델
class ImprovedSeq2Seq(nn.Module):
    """개선된 Seq2Seq 모델"""

    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

        # 인코더와 디코더의 은닉 상태 크기가 다를 경우를 위한 변환층
        if encoder.hidden_size != decoder.hidden_size:
            self.hidden_projection = nn.Linear(encoder.hidden_size, decoder.hidden_size)
            self.cell_projection = nn.Linear(encoder.hidden_size, decoder.hidden_size)
        else:
            self.hidden_projection = None
            self.cell_projection = None

    def forward(self, source, target, teacher_forcing_ratio=0.8):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.vocab_size

        # 인코딩
        encoder_outputs, (hidden, cell), encoder_mask = self.encoder(source)

        # 인코더의 마지막 은닉 상태를 디코더 초기 상태로 사용
        if self.hidden_projection is not None:
            hidden = self.hidden_projection(hidden)
            cell = self.cell_projection(cell)

        # 결과 저장
        outputs = torch.zeros(batch_size, target_len, target_vocab_size)
        attention_weights_list = []

        # 첫 번째 입력은 <SOS> 토큰
        input_token = target[:, 0:1]

        for i in range(1, target_len):
            output, hidden, cell, attention_weights = self.decoder(
                input_token, hidden, cell, encoder_outputs, encoder_mask
            )

            outputs[:, i:i+1] = output
            attention_weights_list.append(attention_weights)

            # Teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(-1)
            input_token = target[:, i:i+1] if teacher_force else top1

        return outputs, attention_weights_list

# 6. 개선된 훈련 함수
def train_improved_model(model, train_data, en_vocab, nl_vocab, num_epochs=200):
    """개선된 모델 훈련"""
    # 더 적극적인 학습률과 스케줄링
    criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=0.002, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    model.train()
    max_len = 10

    # 훈련 루프
    for epoch in range(num_epochs):
        total_loss = 0
        epoch_teacher_forcing = 0.9 - (epoch / num_epochs) * 0.3  # 점진적으로 감소

        # 데이터 셔플
        shuffled_data = train_data.copy()
        random.shuffle(shuffled_data)

        for en_sentence, nl_sentence in shuffled_data:
            # 데이터 준비
            en_indices = sentence_to_indices(en_sentence, en_vocab, max_len)
            nl_words = sentence_to_indices(nl_sentence, nl_vocab, max_len-1)

            nl_input = [nl_vocab.word2idx['<SOS>']] + nl_words[:-1]
            nl_target = nl_words + [nl_vocab.word2idx['<EOS>']]

            # 길이 조정
            nl_input = nl_input[:max_len]
            nl_target = nl_target[:max_len]

            while len(nl_input) < max_len:
                nl_input.append(nl_vocab.word2idx['<PAD>'])
            while len(nl_target) < max_len:
                nl_target.append(nl_vocab.word2idx['<PAD>'])

            en_tensor = torch.LongTensor([en_indices])
            nl_tensor = torch.LongTensor([nl_input])
            target_tensor = torch.LongTensor([nl_target])

            # 순전파
            optimizer.zero_grad()
            output, attention_weights = model(en_tensor, nl_tensor, epoch_teacher_forcing)

            # 손실 계산
            output = output.reshape(-1, output.shape[-1])
            target_tensor = target_tensor.reshape(-1)
            loss = criterion(output, target_tensor)

            # 역전파
            loss.backward()

            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

            optimizer.step()
            total_loss += loss.item()

        scheduler.step()

        if (epoch + 1) % 25 == 0:
            avg_loss = total_loss / len(train_data)
            current_lr = scheduler.get_last_lr()[0]
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, LR: {current_lr:.6f}')

# 7. 번역 및 시각화 함수 (동일)
def translate_improved(model, sentence, en_vocab, nl_vocab, max_len=10):
    """개선된 모델로 번역"""
    model.eval()

    with torch.no_grad():
        en_indices = sentence_to_indices(sentence, en_vocab, max_len)
        en_tensor = torch.LongTensor([en_indices])

        encoder_outputs, (hidden, cell), encoder_mask = model.encoder(en_tensor)

        # 은닉 상태 변환
        if model.hidden_projection is not None:
            hidden = model.hidden_projection(hidden)
            cell = model.cell_projection(cell)

        result = []
        input_token = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

        for _ in range(max_len):
            output, hidden, cell, attention_weights = model.decoder(
                input_token, hidden, cell, encoder_outputs, encoder_mask
            )
            predicted_id = output.argmax(-1).item()

            if predicted_id == nl_vocab.word2idx['<EOS>']:
                break
            if predicted_id == nl_vocab.word2idx['<PAD>']:
                break

            word = nl_vocab.idx2word[predicted_id]
            result.append(word)
            input_token = torch.LongTensor([[predicted_id]])

        return ' '.join(result)

def get_attention_weights(model, sentence, en_vocab, nl_vocab, max_len=10):
    """Attention 가중치 반환 (시각화 없이)"""
    model.eval()

    with torch.no_grad():
        en_indices = sentence_to_indices(sentence, en_vocab, max_len)
        en_tensor = torch.LongTensor([en_indices])

        encoder_outputs, (hidden, cell), encoder_mask = model.encoder(en_tensor)

        if model.hidden_projection is not None:
            hidden = model.hidden_projection(hidden)
            cell = model.cell_projection(cell)

        result_words = []
        attention_matrix = []
        input_token = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

        for _ in range(max_len):
            output, hidden, cell, attention_weights = model.decoder(
                input_token, hidden, cell, encoder_outputs, encoder_mask
            )

            predicted_id = output.argmax(-1).item()

            if predicted_id == nl_vocab.word2idx['<EOS>']:
                break
            if predicted_id == nl_vocab.word2idx['<PAD>']:
                break

            word = nl_vocab.idx2word[predicted_id]
            result_words.append(word)
            attention_matrix.append(attention_weights.squeeze().cpu().numpy())

            input_token = torch.LongTensor([[predicted_id]])

        # 입력 단어들
        input_words = []
        for idx in en_indices:
            if idx == en_vocab.word2idx['<PAD>']:
                break
            input_words.append(en_vocab.idx2word[idx])

        return ' '.join(result_words), attention_matrix, input_words, result_words

# 8. 성능 비교를 위한 기본 Seq2Seq 모델
class BasicSeq2Seq(nn.Module):
    """기본 Seq2Seq 모델 (비교용)"""

    def __init__(self, input_vocab_size, output_vocab_size, embed_size, hidden_size):
        super().__init__()

        # 인코더
        self.encoder_embedding = nn.Embedding(input_vocab_size, embed_size, padding_idx=0)
        self.encoder_lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)

        # 디코더
        self.decoder_embedding = nn.Embedding(output_vocab_size, embed_size, padding_idx=0)
        self.decoder_lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.output_projection = nn.Linear(hidden_size, output_vocab_size)

        self.dropout = nn.Dropout(0.1)

    def forward(self, source, target, teacher_forcing_ratio=0.8):
        batch_size = source.size(0)
        target_len = target.size(1)
        vocab_size = self.output_projection.out_features

        # 인코딩
        embedded_src = self.dropout(self.encoder_embedding(source))
        _, (hidden, cell) = self.encoder_lstm(embedded_src)

        # 디코딩
        outputs = torch.zeros(batch_size, target_len, vocab_size)
        input_token = target[:, 0:1]

        for i in range(1, target_len):
            embedded_tgt = self.dropout(self.decoder_embedding(input_token))
            output, (hidden, cell) = self.decoder_lstm(embedded_tgt, (hidden, cell))
            prediction = self.output_projection(output)
            outputs[:, i:i+1] = prediction

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = prediction.argmax(-1)
            input_token = target[:, i:i+1] if teacher_force else top1

        return outputs

def train_basic_model(model, train_data, en_vocab, nl_vocab, num_epochs=200):
    """기본 모델 훈련"""
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    max_len = 10

    for epoch in range(num_epochs):
        total_loss = 0

        for en_sentence, nl_sentence in train_data:
            en_indices = sentence_to_indices(en_sentence, en_vocab, max_len)
            nl_words = sentence_to_indices(nl_sentence, nl_vocab, max_len-1)

            nl_input = [nl_vocab.word2idx['<SOS>']] + nl_words[:-1]
            nl_target = nl_words + [nl_vocab.word2idx['<EOS>']]

            nl_input = nl_input[:max_len]
            nl_target = nl_target[:max_len]

            while len(nl_input) < max_len:
                nl_input.append(nl_vocab.word2idx['<PAD>'])
            while len(nl_target) < max_len:
                nl_target.append(nl_vocab.word2idx['<PAD>'])

            en_tensor = torch.LongTensor([en_indices])
            nl_tensor = torch.LongTensor([nl_input])
            target_tensor = torch.LongTensor([nl_target])

            optimizer.zero_grad()
            output = model(en_tensor, nl_tensor)

            output = output.reshape(-1, output.shape[-1])
            target_tensor = target_tensor.reshape(-1)
            loss = criterion(output, target_tensor)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            avg_loss = total_loss / len(train_data)
            print(f'Basic Model - Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

def translate_basic(model, sentence, en_vocab, nl_vocab, max_len=10):
    """기본 모델로 번역"""
    model.eval()

    with torch.no_grad():
        en_indices = sentence_to_indices(sentence, en_vocab, max_len)
        en_tensor = torch.LongTensor([en_indices])

        embedded_src = model.encoder_embedding(en_tensor)
        _, (hidden, cell) = model.encoder_lstm(embedded_src)

        result = []
        input_token = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

        for _ in range(max_len):
            embedded_tgt = model.decoder_embedding(input_token)
            output, (hidden, cell) = model.decoder_lstm(embedded_tgt, (hidden, cell))
            prediction = model.output_projection(output)
            predicted_id = prediction.argmax(-1).item()

            if predicted_id == nl_vocab.word2idx['<EOS>']:
                break
            if predicted_id == nl_vocab.word2idx['<PAD>']:
                break

            word = nl_vocab.idx2word[predicted_id]
            result.append(word)
            input_token = torch.LongTensor([[predicted_id]])

        return ' '.join(result)

# 9. 메인 실행 함수
def main():
    print("=" * 70)
    print("기본 Seq2Seq vs 개선된 Attention Seq2Seq 성능 비교")
    print("=" * 70)

    # 데이터 준비
    english_sentences, dutch_sentences = create_datasets()
    en_vocab, nl_vocab = preprocess_data(english_sentences, dutch_sentences)
    train_data = list(zip(english_sentences, dutch_sentences))

    print(f"영어 어휘 수: {len(en_vocab)}")
    print(f"네덜란드어 어휘 수: {len(nl_vocab)}")
    print(f"훈련 데이터 수: {len(train_data)}")
    print()

    # 모델 하이퍼파라미터
    embed_size = 128
    hidden_size = 256

    # 1. 기본 Seq2Seq 모델 훈련
    print("1. 기본 Seq2Seq 모델 훈련...")
    basic_model = BasicSeq2Seq(len(en_vocab), len(nl_vocab), embed_size, hidden_size)
    train_basic_model(basic_model, train_data, en_vocab, nl_vocab, num_epochs=200)
    print()

    # 2. 개선된 Attention 모델 훈련
    print("2. 개선된 Attention Seq2Seq 모델 훈련...")
    encoder = ImprovedEncoder(len(en_vocab), embed_size, hidden_size, dropout=0.1)
    decoder = ImprovedDecoder(len(nl_vocab), embed_size, hidden_size, dropout=0.1)
    attention_model = ImprovedSeq2Seq(encoder, decoder)
    train_improved_model(attention_model, train_data, en_vocab, nl_vocab, num_epochs=200)
    print()

    # 3. 성능 비교
    print("=" * 50)
    print("번역 성능 비교")
    print("=" * 50)

    test_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "I study machine learning"
    ]

    for sentence in test_sentences:
        basic_translation = translate_basic(basic_model, sentence, en_vocab, nl_vocab)
        attention_translation = translate_improved(attention_model, sentence, en_vocab, nl_vocab)

        print(f"입력: {sentence}")
        print(f"기본 모델: {basic_translation}")
        print(f"Attention 모델: {attention_translation}")
        print()

    # 4. 주요 문장 Attention 분석 (텍스트로만)
    print("=" * 50)
    print("'I love llamas' Attention 분석")
    print("=" * 50)

    target_sentence = "I love llamas"
    translation, attention_matrix, input_words, result_words = get_attention_weights(
        attention_model, target_sentence, en_vocab, nl_vocab
    )
    print(f"번역 결과: {translation}")
    print(f"정답: Ik hou van lama's")

    # Attention 가중치를 텍스트로 출력
    if attention_matrix and result_words:
        print(f"\nAttention 가중치 분석:")
        attention_matrix = np.array(attention_matrix)
        attention_matrix = attention_matrix[:, :len(input_words)]

        for i, target_word in enumerate(result_words):
            print(f"\n'{target_word}' 생성 시:")
            for j, source_word in enumerate(input_words):
                weight = attention_matrix[i, j]
                bar = '█' * int(weight * 20)
                print(f"  {source_word:8}: {weight:.3f} {bar}")

    return basic_model, attention_model, en_vocab, nl_vocab

if __name__ == "__main__":
    basic_model, attention_model, en_vocab, nl_vocab = main()

    print("\n" + "=" * 50)
    print("개선사항 요약")
    print("=" * 50)
    print("1. Luong Attention으로 변경 (더 효율적)")
    print("2. 단방향 LSTM 사용 (안정성 향상)")
    print("3. 개선된 학습률 스케줄링 (CosineAnnealingLR)")
    print("4. Label Smoothing 적용 (일반화 향상)")
    print("5. 그래디언트 클리핑 강화 (안정성 향상)")
    print("6. Teacher Forcing 점진적 감소")
    print("7. 더 나은 정규화 (AdamW + Weight Decay)")

    print("\n예상 개선 효과:")
    print("- 번역 정확도 향상")
    print("- 더 자연스러운 문장 생성")
    print("- 안정적인 훈련 과정")
    print("- 명확한 Attention 패턴")

기본 Seq2Seq vs 개선된 Attention Seq2Seq 성능 비교
영어 어휘 수: 25
네덜란드어 어휘 수: 26
훈련 데이터 수: 10

1. 기본 Seq2Seq 모델 훈련...
Basic Model - Epoch [50/200], Loss: 0.7354
Basic Model - Epoch [100/200], Loss: 0.7338
Basic Model - Epoch [150/200], Loss: 0.7334
Basic Model - Epoch [200/200], Loss: 0.7333

2. 개선된 Attention Seq2Seq 모델 훈련...
Epoch [25/200], Loss: 1.2385, LR: 0.001924
Epoch [50/200], Loss: 1.2241, LR: 0.001707
Epoch [75/200], Loss: 1.2199, LR: 0.001383
Epoch [100/200], Loss: 1.2194, LR: 0.001000
Epoch [125/200], Loss: 1.2191, LR: 0.000617
Epoch [150/200], Loss: 1.2189, LR: 0.000293
Epoch [175/200], Loss: 1.2187, LR: 0.000076
Epoch [200/200], Loss: 1.2187, LR: 0.000000

번역 성능 비교
입력: I love llamas
기본 모델: hou van lama's lama's lama's
Attention 모델: hou van lama's lama's lama's

입력: I like cats
기본 모델: hou van katten katten katten
Attention 모델: hou van katten katten

입력: I eat apples
기본 모델: eet appels appels appels appels
Attention 모델: eet appels learning

입력: I drink water
기본 모델: drink water water wate



---



## **3. Transformer 아키텍처 적용**

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import math
import random
from collections import Counter

# 시드 설정으로 재현 가능한 결과 보장
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 1. 데이터 준비 및 전처리 (기존과 동일)
class Vocabulary:
    def __init__(self):
        self.word2idx = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
        self.idx2word = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>'}
        self.word_count = 4

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.word_count
            self.idx2word[self.word_count] = word
            self.word_count += 1

    def __len__(self):
        return self.word_count

def create_extended_datasets():
    """확장된 영어-네덜란드어 번역 데이터셋"""
    english_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "I read books",
        "I watch movies",
        "I play games",
        "I write code",
        "I love programming",
        "I study machine learning",
        "I enjoy reading books",
        "I prefer tea over coffee",
        "I want to travel",
        "I need some help",
        "I can speak English",
        "You are very kind",
        "We learn together",
        "They work hard",
        "She loves music",
        "He reads newspapers"
    ]

    dutch_sentences = [
        "Ik hou van lama's",
        "Ik hou van katten",
        "Ik eet appels",
        "Ik drink water",
        "Ik lees boeken",
        "Ik kijk naar films",
        "Ik speel spelletjes",
        "Ik schrijf code",
        "Ik hou van programmeren",
        "Ik studeer machine learning",
        "Ik geniet van het lezen van boeken",
        "Ik verkies thee boven koffie",
        "Ik wil reizen",
        "Ik heb wat hulp nodig",
        "Ik kan Engels spreken",
        "Je bent erg aardig",
        "We leren samen",
        "Ze werken hard",
        "Ze houdt van muziek",
        "Hij leest kranten"
    ]

    return english_sentences, dutch_sentences

def preprocess_data(english_sentences, dutch_sentences):
    """데이터 전처리 및 어휘 사전 생성"""
    en_vocab = Vocabulary()
    nl_vocab = Vocabulary()

    for sentence in english_sentences:
        for word in sentence.lower().split():
            en_vocab.add_word(word)

    for sentence in dutch_sentences:
        for word in sentence.lower().split():
            nl_vocab.add_word(word)

    return en_vocab, nl_vocab

def sentence_to_indices(sentence, vocab, max_len=12):
    """문장을 인덱스 시퀀스로 변환"""
    words = sentence.lower().split()
    indices = [vocab.word2idx.get(word, vocab.word2idx['<UNK>']) for word in words]

    if len(indices) >= max_len:
        indices = indices[:max_len-1]

    while len(indices) < max_len:
        indices.append(vocab.word2idx['<PAD>'])

    return indices

# 2. Positional Encoding 구현
class PositionalEncoding(nn.Module):
    """Transformer의 위치 인코딩"""

    def __init__(self, d_model, max_seq_length=1000):
        super().__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
        # PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                           (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # [1, max_seq_length, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: [batch_size, seq_length, d_model]
        seq_length = x.size(1)
        return x + self.pe[:, :seq_length, :]


# 3. Multi-Head Attention 구현
class MultiHeadAttention(nn.Module):
    """Multi-Head Self-Attention 메커니즘"""

    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Q, K, V를 위한 선형 변환
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """Scaled Dot-Product Attention"""
        # Q: [batch_size, num_heads, query_len, d_k]
        # K: [batch_size, num_heads, key_len, d_k]
        # V: [batch_size, num_heads, value_len, d_k]

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # scores: [batch_size, num_heads, query_len, key_len]

        if mask is not None:
            # 마스크 차원 조정
            if mask.dim() == 2:  # [batch_size, key_len] - padding mask
                mask = mask.unsqueeze(1).unsqueeze(1)  # [batch_size, 1, 1, key_len]
            elif mask.dim() == 3:  # [batch_size, query_len, key_len] - causal mask
                mask = mask.unsqueeze(1)  # [batch_size, 1, query_len, key_len]
            elif mask.dim() == 4:  # [batch_size, 1, query_len, key_len] - 이미 맞음
                pass

            # Boolean 마스크를 사용하여 마스킹
            mask_value = -1e9
            scores = scores.masked_fill(~mask, mask_value)

        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        context = torch.matmul(attention_weights, V)
        # context: [batch_size, num_heads, query_len, d_k]
        return context, attention_weights

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        query_len = query.size(1)
        key_len = key.size(1)
        value_len = value.size(1)

        # 1. Q, K, V 계산
        Q = self.W_q(query)  # [batch_size, query_len, d_model]
        K = self.W_k(key)    # [batch_size, key_len, d_model]
        V = self.W_v(value)  # [batch_size, value_len, d_model]

        # 2. Multi-head로 분할 (각각의 시퀀스 길이 사용)
        Q = Q.view(batch_size, query_len, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, key_len, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, value_len, self.num_heads, self.d_k).transpose(1, 2)
        # 결과: Q[batch_size, num_heads, query_len, d_k], K[batch_size, num_heads, key_len, d_k]

        # 3. Scaled Dot-Product Attention
        context, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)

        # 4. Multi-head 결과 연결
        context = context.transpose(1, 2).contiguous().view(
            batch_size, query_len, self.d_model
        )

        # 5. 최종 선형 변환
        output = self.W_o(context)

        return output, attention_weights


# 4. Feed-Forward Network
class FeedForward(nn.Module):
    """Position-wise Feed-Forward Network"""

    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # FFN(x) = max(0, xW1 + b1)W2 + b2
        return self.linear2(self.dropout(F.relu(self.linear1(x))))


# 5. Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    """Transformer 인코더 레이어"""

    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # 1. Self-Attention + Residual Connection + Layer Norm
        attn_output, _ = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))

        # 2. Feed-Forward + Residual Connection + Layer Norm
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))

        return x

# 6. Transformer Decoder Layer
class TransformerDecoderLayer(nn.Module):
    """Transformer 디코더 레이어"""

    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        # 1. Self-Attention (with causal mask)
        attn_output, _ = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))

        # 2. Cross-Attention (Encoder-Decoder Attention)
        cross_attn_output, cross_attn_weights = self.cross_attention(
            x, encoder_output, encoder_output, src_mask
        )
        x = self.norm2(x + self.dropout(cross_attn_output))

        # 3. Feed-Forward
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))

        return x, cross_attn_weights


# 7. 완전한 Transformer 모델
class Transformer(nn.Module):
    """완전한 Transformer 모델"""

    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=256, num_heads=8,
                 num_encoder_layers=3, num_decoder_layers=3, d_ff=1024,
                 max_seq_length=100, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size

        # 임베딩 레이어
        self.src_embedding = nn.Embedding(src_vocab_size, d_model, padding_idx=0)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model, padding_idx=0)

        # 위치 인코딩
        self.pos_encoding = PositionalEncoding(d_model, max_seq_length)

        # 인코더
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_encoder_layers)
        ])

        # 디코더
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_decoder_layers)
        ])

        # 출력 투영
        self.output_projection = nn.Linear(d_model, tgt_vocab_size)

        self.dropout = nn.Dropout(dropout)

        # 가중치 초기화
        self._init_weights()

    def _init_weights(self):
        """가중치 초기화"""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def create_padding_mask(self, x, pad_idx=0):
        """패딩 마스크 생성"""
        return (x != pad_idx)  # Boolean 타입으로 반환

    def create_causal_mask(self, size):
        """인과적 마스크 생성 (미래 토큰 보지 못하게)"""
        mask = torch.triu(torch.ones(size, size), diagonal=1)
        return (mask == 0)  # Boolean 타입으로 반환

    def encode(self, src, src_mask):
        """인코더 forward pass"""
        # 임베딩 + 위치 인코딩
        x = self.src_embedding(src) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # 인코더 레이어들 통과
        for layer in self.encoder_layers:
            x = layer(x, src_mask)

        return x

    def decode(self, tgt, encoder_output, src_mask, tgt_mask):
        """디코더 forward pass"""
        # 임베딩 + 위치 인코딩
        x = self.tgt_embedding(tgt) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        cross_attentions = []

        # 디코더 레이어들 통과
        for layer in self.decoder_layers:
            x, cross_attn = layer(x, encoder_output, src_mask, tgt_mask)
            cross_attentions.append(cross_attn)

        return x, cross_attentions

    def forward(self, src, tgt, teacher_forcing_ratio=1.0):
        """전체 forward pass"""
        batch_size, tgt_seq_len = tgt.size()

        # 마스크 생성 (Boolean 타입)
        src_mask = self.create_padding_mask(src)  # [batch_size, src_len]
        tgt_padding_mask = self.create_padding_mask(tgt)  # [batch_size, tgt_len]

        # 인과적 마스크 생성
        causal_mask = self.create_causal_mask(tgt_seq_len).to(tgt.device)  # [tgt_len, tgt_len]

        # 타겟 마스크: 패딩 마스크와 인과적 마스크 결합
        # Broadcasting을 통해 올바르게 결합
        tgt_mask = tgt_padding_mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, tgt_len]
        causal_mask = causal_mask.unsqueeze(0).unsqueeze(0)   # [1, 1, tgt_len, tgt_len]
        tgt_mask = tgt_mask & causal_mask  # Broadcasting으로 [batch_size, 1, tgt_len, tgt_len]

        # 인코딩
        encoder_output = self.encode(src, src_mask)

        # 디코딩 (Teacher Forcing 사용)
        tgt_input = tgt[:, :-1]  # 마지막 토큰 제외
        decoder_output, cross_attentions = self.decode(
            tgt_input, encoder_output, src_mask,
            tgt_mask[:, :, :-1, :-1]  # 크기 맞춤
        )

        # 출력 투영
        output = self.output_projection(decoder_output)

        return output, cross_attentions

# 8. 훈련 함수
# def train_transformer(model, train_data, en_vocab, nl_vocab, num_epochs=300):
#     """Transformer 모델 훈련"""
#     criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)
#     optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01,
#                            betas=(0.9, 0.98), eps=1e-9)

#     # 더 간단한 스케줄러
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

#     model.train()
#     max_len = 12

#     for epoch in range(num_epochs):
#         total_loss = 0

#         # 데이터 셔플
#         shuffled_data = train_data.copy()
#         random.shuffle(shuffled_data)

#         for batch_idx, (en_sentence, nl_sentence) in enumerate(shuffled_data):
#             try:
#                 # 데이터 준비 - 길이 엄격하게 관리
#                 en_indices = sentence_to_indices(en_sentence, en_vocab, max_len)
#                 nl_indices = sentence_to_indices(nl_sentence, nl_vocab, max_len-2)  # SOS, EOS 공간 확보

#                 # 타겟 시퀀스 구성: <SOS> + 문장 + <EOS>
#                 nl_input = [nl_vocab.word2idx['<SOS>']] + nl_indices
#                 nl_target = nl_indices + [nl_vocab.word2idx['<EOS>']]

#                 # 정확히 max_len으로 맞추기
#                 if len(nl_input) > max_len:
#                     nl_input = nl_input[:max_len]
#                 if len(nl_target) > max_len:
#                     nl_target = nl_target[:max_len]

#                 # 패딩 추가
#                 while len(nl_input) < max_len:
#                     nl_input.append(nl_vocab.word2idx['<PAD>'])
#                 while len(nl_target) < max_len:
#                     nl_target.append(nl_vocab.word2idx['<PAD>'])

#                 # 텐서 변환
#                 src_tensor = torch.LongTensor([en_indices])
#                 tgt_tensor = torch.LongTensor([nl_input])
#                 target_tensor = torch.LongTensor([nl_target])

#                 # 길이 검증
#                 assert src_tensor.size(1) == max_len, f"Source length mismatch: {src_tensor.size(1)} != {max_len}"
#                 assert tgt_tensor.size(1) == max_len, f"Target input length mismatch: {tgt_tensor.size(1)} != {max_len}"
#                 assert target_tensor.size(1) == max_len, f"Target output length mismatch: {target_tensor.size(1)} != {max_len}"

#                 # 순전파
#                 optimizer.zero_grad()
#                 output, _ = model(src_tensor, tgt_tensor)

#                 # 출력 길이 검증
#                 expected_output_len = max_len - 1  # tgt_tensor에서 마지막 토큰 제외
#                 assert output.size(1) == expected_output_len, f"Output length mismatch: {output.size(1)} != {expected_output_len}"

#                 # 손실 계산 - 차원 맞추기
#                 output_flat = output.reshape(-1, output.shape[-1])  # [batch*(max_len-1), vocab_size]
#                 target_flat = target_tensor.reshape(-1)             # [batch*max_len]

#                 # 타겟도 같은 길이로 맞추기 (첫 번째 토큰 제외)
#                 target_adjusted = target_tensor[:, :expected_output_len].reshape(-1)  # [batch*(max_len-1)]

#                 assert output_flat.size(0) == target_adjusted.size(0), \
#                     f"Batch size mismatch: output {output_flat.size(0)} != target {target_adjusted.size(0)}"

#                 loss = criterion(output_flat, target_adjusted)

#                 # 역전파
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#                 optimizer.step()

#                 total_loss += loss.item()

#             except Exception as e:
#                 print(f"Error in batch {batch_idx}: {e}")
#                 print(f"Sentence lengths - EN: {len(en_sentence.split())}, NL: {len(nl_sentence.split())}")
#                 continue

#         scheduler.step()

#         if (epoch + 1) % 50 == 0:
#             avg_loss = total_loss / len(train_data) if len(train_data) > 0 else 0
#             current_lr = scheduler.get_last_lr()[0]
#             print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, LR: {current_lr:.6f}')

# 8. 훈련 함수 (수정됨)
def train_transformer(model, train_data, en_vocab, nl_vocab, num_epochs=300):
    """Transformer 모델 훈련 (데이터 전처리 로직 수정)"""
    criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01,
                            betas=(0.9, 0.98), eps=1e-9)

    # 더 간단한 스케줄러
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

    model.train()
    max_len = 12 # 이 값(max_len)이 12로 설정되어 있습니다.

    for epoch in range(num_epochs):
        total_loss = 0

        # 데이터 셔플
        shuffled_data = train_data.copy()
        random.shuffle(shuffled_data)

        for batch_idx, (en_sentence, nl_sentence) in enumerate(shuffled_data):
            try:
                # --- START: 수정된 데이터 준비 로직 ---

                # 1. Source (영어) 준비 (기존과 동일)
                #    [word1, word2, ..., <PAD>, <PAD>]
                en_indices = sentence_to_indices(en_sentence, en_vocab, max_len)
                src_tensor = torch.LongTensor([en_indices])

                # 2. Target (네덜란드어) 준비 (버그 수정)

                # (a) 단어로 변환
                nl_words = nl_sentence.lower().split()

                # (b) 인덱스로 변환 (패딩 X)
                nl_indices_raw = [nl_vocab.word2idx.get(w, nl_vocab.word2idx['<UNK>']) for w in nl_words]

                # (c) <SOS>와 <EOS>를 위한 공간(2개) 확보 (max_len-2)
                if len(nl_indices_raw) > max_len - 2:
                    nl_indices_raw = nl_indices_raw[:max_len - 2]

                # (d) <SOS>와 <EOS> 토큰 추가
                # nl_input_list: [<SOS>, w1, w2, ...]
                nl_input_list = [nl_vocab.word2idx['<SOS>']] + nl_indices_raw
                # nl_target_list: [w1, w2, ..., <EOS>]
                nl_target_list = nl_indices_raw + [nl_vocab.word2idx['<EOS>']]

                # (e) <PAD> 토큰으로 max_len까지 채우기
                #    이제 <EOS>가 <PAD> 앞에 정확히 위치합니다.
                while len(nl_input_list) < max_len:
                    nl_input_list.append(nl_vocab.word2idx['<PAD>'])

                while len(nl_target_list) < max_len:
                    nl_target_list.append(nl_vocab.word2idx['<PAD>'])

                # (f) 텐서 변환
                tgt_tensor = torch.LongTensor([nl_input_list])
                target_tensor = torch.LongTensor([nl_target_list])

                # --- END: 수정된 데이터 준비 로직 ---


                # 길이 검증
                assert src_tensor.size(1) == max_len, f"Source length mismatch: {src_tensor.size(1)} != {max_len}"
                assert tgt_tensor.size(1) == max_len, f"Target input length mismatch: {tgt_tensor.size(1)} != {max_len}"
                assert target_tensor.size(1) == max_len, f"Target output length mismatch: {target_tensor.size(1)} != {max_len}"

                # 순전파
                optimizer.zero_grad()
                output, _ = model(src_tensor, tgt_tensor)

                # 출력 길이 검증
                expected_output_len = max_len - 1  # tgt_tensor에서 마지막 토큰 제외
                assert output.size(1) == expected_output_len, f"Output length mismatch: {output.size(1)} != {expected_output_len}"

                # 손실 계산 - 차원 맞추기
                output_flat = output.reshape(-1, output.shape[-1])  # [batch*(max_len-1), vocab_size]

                # 타겟도 같은 길이로 맞추기
                target_adjusted = target_tensor[:, :expected_output_len].reshape(-1)  # [batch*(max_len-1)]

                assert output_flat.size(0) == target_adjusted.size(0), \
                    f"Batch size mismatch: output {output_flat.size(0)} != target {target_adjusted.size(0)}"

                loss = criterion(output_flat, target_adjusted)

                # 역전파
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_loss += loss.item()

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                print(f"Sentence lengths - EN: {len(en_sentence.split())}, NL: {len(nl_sentence.split())}")
                continue

        scheduler.step()

        if (epoch + 1) % 50 == 0:
            avg_loss = total_loss / len(train_data) if len(train_data) > 0 else 0
            current_lr = scheduler.get_last_lr()[0]
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, LR: {current_lr:.6f}')



# 9. 번역 함수 (Greedy Decoding)
def translate_transformer(model, sentence, en_vocab, nl_vocab, max_len=12):
    """Transformer로 번역 (Greedy Decoding)"""
    model.eval()

    with torch.no_grad():
        # 소스 문장 인코딩
        en_indices = sentence_to_indices(sentence, en_vocab, max_len)
        src_tensor = torch.LongTensor([en_indices])

        # 소스 마스크 생성
        src_mask = model.create_padding_mask(src_tensor)

        # 인코더 출력
        encoder_output = model.encode(src_tensor, src_mask)

        # 디코딩 시작
        result = []
        tgt_input = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

        for _ in range(max_len):
            # 현재까지의 타겟 시퀀스로 디코딩
            current_len = tgt_input.size(1)
            causal_mask = model.create_causal_mask(current_len).to(tgt_input.device)

            # 배치 차원 추가
            tgt_self_mask = causal_mask.unsqueeze(0)  # [1, current_len, current_len]

            decoder_output, _ = model.decode(tgt_input, encoder_output, src_mask, tgt_self_mask)

            # 마지막 위치의 출력으로 다음 토큰 예측
            next_token_logits = model.output_projection(decoder_output[:, -1, :])
            predicted_id = next_token_logits.argmax(-1).item()

            # 종료 조건
            if predicted_id == nl_vocab.word2idx['<EOS>']:
                break
            if predicted_id == nl_vocab.word2idx['<PAD>']:
                break

            word = nl_vocab.idx2word[predicted_id]
            result.append(word)

            # 다음 입력에 예측된 토큰 추가
            tgt_input = torch.cat([
                tgt_input,
                torch.LongTensor([[predicted_id]])
            ], dim=1)

        return ' '.join(result)

# 10. Beam Search 디코딩 (고급)
def beam_search_translate(model, sentence, en_vocab, nl_vocab, beam_size=3, max_len=12):
    """Beam Search를 사용한 더 나은 번역"""
    model.eval()

    with torch.no_grad():
        # 소스 인코딩
        en_indices = sentence_to_indices(sentence, en_vocab, max_len)
        src_tensor = torch.LongTensor([en_indices])
        src_mask = model.create_padding_mask(src_tensor)
        encoder_output = model.encode(src_tensor, src_mask)

        # Beam Search 초기화
        beams = [([nl_vocab.word2idx['<SOS>']], 0.0)]  # (sequence, score)

        for _ in range(max_len):
            new_beams = []

            for sequence, score in beams:
                if sequence[-1] == nl_vocab.word2idx['<EOS>']:
                    new_beams.append((sequence, score))
                    continue

                # 현재 시퀀스로 디코딩
                tgt_input = torch.LongTensor([sequence])
                current_len = tgt_input.size(1)
                causal_mask = model.create_causal_mask(current_len).to(tgt_input.device)

                # 배치 차원 추가
                tgt_self_mask = causal_mask.unsqueeze(0)  # [1, current_len, current_len]

                decoder_output, _ = model.decode(tgt_input, encoder_output, src_mask, tgt_self_mask)
                next_token_logits = model.output_projection(decoder_output[:, -1, :])

                # Top-k 토큰 선택
                log_probs = F.log_softmax(next_token_logits, dim=-1)
                top_k_probs, top_k_indices = torch.topk(log_probs, beam_size)

                for i in range(beam_size):
                    token_id = top_k_indices[0, i].item()
                    token_score = top_k_probs[0, i].item()
                    new_sequence = sequence + [token_id]
                    new_score = score + token_score
                    new_beams.append((new_sequence, new_score))

            # 상위 beam_size개 선택
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

            # 모든 빔이 종료되었는지 확인
            if all(seq[-1] == nl_vocab.word2idx['<EOS>'] for seq, _ in beams):
                break

        # 최고 점수의 시퀀스 선택
        best_sequence, _ = max(beams, key=lambda x: x[1])

        # 토큰을 단어로 변환
        result = []
        for token_id in best_sequence[1:]:  # <SOS> 제외
            if token_id == nl_vocab.word2idx['<EOS>']:
                break
            if token_id == nl_vocab.word2idx['<PAD>']:
                break
            result.append(nl_vocab.idx2word[token_id])

        return ' '.join(result)

# 11. Cross-Attention 분석 함수
def analyze_cross_attention(model, sentence, en_vocab, nl_vocab, max_len=12):
    """Cross-Attention 가중치 분석 - 안전한 버전"""
    model.eval()

    with torch.no_grad():
        try:
            # 소스 인코딩
            en_indices = sentence_to_indices(sentence, en_vocab, max_len)
            src_tensor = torch.LongTensor([en_indices])
            src_mask = model.create_padding_mask(src_tensor)
            encoder_output = model.encode(src_tensor, src_mask)

            # 번역하면서 attention 수집
            result_words = []
            attention_matrices = []
            tgt_input = torch.LongTensor([[nl_vocab.word2idx['<SOS>']]])

            # 입력 단어들
            input_words = []
            for idx in en_indices:
                if idx == en_vocab.word2idx['<PAD>']:
                    break
                input_words.append(en_vocab.idx2word[idx])

            for step in range(max_len-1):  # max_len-1로 제한
                current_len = tgt_input.size(1)
                causal_mask = model.create_causal_mask(current_len).to(tgt_input.device)
                tgt_self_mask = causal_mask.unsqueeze(0)

                decoder_output, cross_attentions = model.decode(
                    tgt_input, encoder_output, src_mask, tgt_self_mask
                )

                next_token_logits = model.output_projection(decoder_output[:, -1, :])
                predicted_id = next_token_logits.argmax(-1).item()

                if predicted_id == nl_vocab.word2idx['<EOS>']:
                    break
                if predicted_id == nl_vocab.word2idx['<PAD>']:
                    break

                word = nl_vocab.idx2word[predicted_id]
                result_words.append(word)

                # Cross-attention 분석 (안전하게 처리)
                if cross_attentions and len(cross_attentions) > 0:
                    try:
                        last_cross_attention = cross_attentions[-1]
                        if last_cross_attention is not None and last_cross_attention.size(0) > 0:
                            avg_attention = last_cross_attention.mean(dim=1)
                            if avg_attention.size(1) > 0:  # 시퀀스 길이 확인
                                current_attention = avg_attention[0, -1, :len(input_words)].cpu().numpy()
                                attention_matrices.append(current_attention)
                    except Exception as e:
                        print(f"Attention extraction error at step {step}: {e}")

                tgt_input = torch.cat([
                    tgt_input,
                    torch.LongTensor([[predicted_id]])
                ], dim=1)

            return ' '.join(result_words), attention_matrices, input_words, result_words

        except Exception as e:
            print(f"Error in attention analysis: {e}")
            return sentence, [], [], []

# 12. 메인 실행 함수 - 안전한 버전
def main():
    """메인 실행 함수"""
    print("=" * 70)
    print("Transformer 아키텍처 영어-네덜란드어 번역 모델")
    print("=" * 70)

    # 데이터 준비
    english_sentences, dutch_sentences = create_extended_datasets()
    en_vocab, nl_vocab = preprocess_data(english_sentences, dutch_sentences)
    train_data = list(zip(english_sentences, dutch_sentences))

    print(f"영어 어휘 수: {len(en_vocab)}")
    print(f"네덜란드어 어휘 수: {len(nl_vocab)}")
    print(f"훈련 데이터 수: {len(train_data)}")
    print()

    # 데이터 길이 분석
    print("데이터 길이 분석:")
    max_en_len = max(len(s.split()) for s in english_sentences)
    max_nl_len = max(len(s.split()) for s in dutch_sentences)
    print(f"최대 영어 문장 길이: {max_en_len}")
    print(f"최대 네덜란드어 문장 길이: {max_nl_len}")
    print()

    # Transformer 모델 생성 (더 작은 크기로 안정성 확보)
    model = Transformer(
        src_vocab_size=len(en_vocab),
        tgt_vocab_size=len(nl_vocab),
        d_model=128,          # 256 → 128 (메모리 절약)
        num_heads=4,          # 8 → 4 (계산 단순화)
        num_encoder_layers=2, # 3 → 2 (안정성 향상)
        num_decoder_layers=2, # 3 → 2 (안정성 향상)
        d_ff=512,            # 1024 → 512 (메모리 절약)
        dropout=0.1
    )

    print("Transformer 모델 구조:")
    print(f"  임베딩 차원: 128")
    print(f"  Attention 헤드 수: 4")
    print(f"  인코더 레이어: 2")
    print(f"  디코더 레이어: 2")
    print(f"  Feed-Forward 차원: 512")
    total_params = sum(p.numel() for p in model.parameters())
    print(f"  총 파라미터 수: {total_params:,}")
    print()

    # 모델 훈련 (더 적은 에포크로 안정성 확보)
    print("Transformer 모델 훈련 시작...")
    try:
        train_transformer(model, train_data, en_vocab, nl_vocab, num_epochs=150)
        print("훈련 완료!")
    except Exception as e:
        print(f"훈련 중 오류 발생: {e}")
        print("더 간단한 설정으로 재시도...")
        # 더 간단한 모델로 재시도
        model = Transformer(
            src_vocab_size=len(en_vocab),
            tgt_vocab_size=len(nl_vocab),
            d_model=64,
            num_heads=2,
            num_encoder_layers=1,
            num_decoder_layers=1,
            d_ff=256,
            dropout=0.1
        )
        train_transformer(model, train_data, en_vocab, nl_vocab, num_epochs=100)
    print()

    # 번역 테스트
    print("=" * 50)
    print("번역 성능 테스트")
    print("=" * 50)

    test_sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "You are very kind"
    ]

    print("Greedy Decoding 결과:")
    print("-" * 30)
    for sentence in test_sentences:
        try:
            translation = translate_transformer(model, sentence, en_vocab, nl_vocab)
            print(f"영어: {sentence}")
            print(f"번역: {translation}")
            print()
        except Exception as e:
            print(f"번역 오류 ({sentence}): {e}")
            continue

    # Beam Search 테스트 (선택적)
    print("Beam Search 결과:")
    print("-" * 30)
    for sentence in test_sentences[:3]:  # 처음 3개만
        try:
            translation = beam_search_translate(model, sentence, en_vocab, nl_vocab, beam_size=2)
            print(f"영어: {sentence}")
            print(f"번역: {translation}")
            print()
        except Exception as e:
            print(f"Beam search 오류 ({sentence}): {e}")
            continue

    # Cross-Attention 분석
    print("=" * 50)
    print("'I love llamas' Cross-Attention 분석")
    print("=" * 50)

    target_sentence = "I love llamas"
    try:
        translation, attention_matrices, input_words, result_words = analyze_cross_attention(
            model, target_sentence, en_vocab, nl_vocab
        )

        print(f"입력: {target_sentence}")
        print(f"번역: {translation}")
        print(f"정답: Ik hou van lama's")
        print()

        if attention_matrices and result_words and len(attention_matrices) > 0:
            print("Cross-Attention 가중치 (Encoder→Decoder):")
            for i, (target_word, attention) in enumerate(zip(result_words, attention_matrices)):
                if len(attention) == len(input_words):
                    print(f"\n'{target_word}' 생성 시 주목하는 영어 단어:")
                    for j, (source_word, weight) in enumerate(zip(input_words, attention)):
                        bar = '█' * int(weight * 20)
                        print(f"  {source_word:8}: {weight:.3f} {bar}")
        else:
            print("Attention 분석 결과를 가져올 수 없습니다.")

    except Exception as e:
        print(f"Attention 분석 오류: {e}")


    return model, en_vocab, nl_vocab


#  성능 평가 및 분석
def evaluate_model_performance(model, en_vocab, nl_vocab):
    """모델 성능 정량적 평가"""
    print("\n" + "=" * 50)
    print("모델 성능 정량적 평가")
    print("=" * 50)

    test_cases = [
        ("I love llamas", "Ik hou van lama's"),
        ("I like cats", "Ik hou van katten"),
        ("I eat apples", "Ik eet appels"),
        ("I drink water", "Ik drink water"),
        ("You are very kind", "Je bent erg aardig")
    ]

    correct_greedy = 0
    correct_beam = 0

    print("번역 정확도 평가:")
    print("문장" + " " * 15 + "| Greedy | Beam Search")
    print("-" * 50)

    for en_sentence, correct_nl in test_cases:
        greedy_result = translate_transformer(model, en_sentence, en_vocab, nl_vocab)
        beam_result = beam_search_translate(model, en_sentence, en_vocab, nl_vocab)

        greedy_correct = greedy_result.strip().lower() == correct_nl.lower()
        beam_correct = beam_result.strip().lower() == correct_nl.lower()

        if greedy_correct:
            correct_greedy += 1
        if beam_correct:
            correct_beam += 1

        print(f"{en_sentence:<20} | {'✓' if greedy_correct else '✗':^6} | {'✓' if beam_correct else '✗':^11}")

    greedy_accuracy = (correct_greedy / len(test_cases)) * 100
    beam_accuracy = (correct_beam / len(test_cases)) * 100

    print("-" * 50)
    print(f"Greedy Decoding 정확도: {greedy_accuracy:.1f}%")
    print(f"Beam Search 정확도: {beam_accuracy:.1f}%")
    print(f"Beam Search 개선도: +{beam_accuracy - greedy_accuracy:.1f}%")

# 실행부
if __name__ == "__main__":
    # 메인 모델 훈련 및 테스트
    model, en_vocab, nl_vocab = main()


    # 성능 평가
    evaluate_model_performance(model, en_vocab, nl_vocab)

    print("\n" + "-" * 30)
    print("Transformer 모델 학습 완료!")
    print("주요 개선사항:")
    print("• 병렬 처리로 빠른 훈련")
    print("• Multi-Head Attention으로 풍부한 표현")
    print("• Self-Attention으로 장거리 의존성 해결")
    print("• Beam Search로 더 정확한 번역")
    print("-" * 30)

Transformer 아키텍처 영어-네덜란드어 번역 모델
영어 어휘 수: 56
네덜란드어 어휘 수: 57
훈련 데이터 수: 20

데이터 길이 분석:
최대 영어 문장 길이: 5
최대 네덜란드어 문장 길이: 7

Transformer 모델 구조:
  임베딩 차원: 128
  Attention 헤드 수: 4
  인코더 레이어: 2
  디코더 레이어: 2
  Feed-Forward 차원: 512
  총 파라미터 수: 947,513

Transformer 모델 훈련 시작...
Epoch [50/150], Loss: 0.8358, LR: 0.000100
Epoch [100/150], Loss: 0.7775, LR: 0.000050
Epoch [150/150], Loss: 0.7538, LR: 0.000050
훈련 완료!

번역 성능 테스트
Greedy Decoding 결과:
------------------------------
영어: I love llamas
번역: ik hou van lama's

영어: I like cats
번역: ik hou van katten

영어: I eat apples
번역: ik eet appels

영어: I drink water
번역: ik drink water

영어: You are very kind
번역: je bent erg aardig

Beam Search 결과:
------------------------------
영어: I love llamas
번역: ik hou van lama's

영어: I like cats
번역: ik hou van katten

영어: I eat apples
번역: ik eet appels

'I love llamas' Cross-Attention 분석
입력: I love llamas
번역: ik hou van lama's
정답: Ik hou van lama's

Cross-Attention 가중치 (Encoder→Decoder):

'ik' 생성 시 주목하는 영어 단어:
  i       : 



---



## **4. LLM모델(사전학습모델) 사용**

In [8]:
# 간단한 영어-네덜란드어 번역기
# 코랩에서 먼저 실행: !pip install transformers torch

from transformers import pipeline
import torch

def simple_translator():
    # 번역 파이프라인 로드
    print("모델 로딩 중...")
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-en-nl",
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ 로딩 완료!")
    print()

    # 테스트 문장들
    sentences = [
        "I love llamas",
        "I like cats",
        "I eat apples",
        "I drink water",
        "You are very kind"
    ]

    # 번역 실행
    print("번역 결과:")
    print("-" * 20)
    for sentence in sentences:
        result = translator(sentence)
        translation = result[0]['translation_text']
        print(f"영어: {sentence}")
        print(f"네덜란드어: {translation}")
        print()

    return translator

def translate_target_sentence(translator):
    target_sentence = "I love llamas"
    result = translator(target_sentence)
    translation = result[0]['translation_text']

    print(f"영어: {target_sentence}")
    print(f"네덜란드어: {translation}")
    print(f"정답: Ik hou van lama's")

    # 정확도 확인
    if "hou van" in translation.lower():
        print("결과: ✅ 완벽한 번역!")
    else:
        print("결과: ⚠️ 부분적 일치")

    print()

# 메인 실행
if __name__ == "__main__":
    # 기본 번역 테스트
    translator = simple_translator()

    # 'I love llamas' 특별 번역
    translate_target_sentence(translator)

    print("번역 완료! 🎉")

모델 로딩 중...


Device set to use cuda:0


✅ 로딩 완료!

번역 결과:
--------------------
영어: I love llamas
네덜란드어: Ik hou van lama's.

영어: I like cats
네덜란드어: Ik hou van katten.

영어: I eat apples
네덜란드어: Ik eet appels.

영어: I drink water
네덜란드어: Ik drink water.

영어: You are very kind
네덜란드어: U bent erg aardig.

영어: I love llamas
네덜란드어: Ik hou van lama's.
정답: Ik hou van lama's
결과: ✅ 완벽한 번역!

번역 완료! 🎉
